#### Next two cells are only needed for a Google Colab environment.

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd '/content/drive/My Drive/CodingProjects/skateboard_trick_classification'

In [0]:
import numpy as np

from keras.applications.densenet import DenseNet121
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Dropout, Softmax
from keras.models import load_model, Model
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

from utils import config

## Data Generators

In [19]:
data_generator = ImageDataGenerator(rescale=1/255.0)

training_generator = data_generator.flow_from_directory(
    config.SPECTROGRAM_TRAINING_DIR,
    target_size=(config.SPECTROGRAM_HEIGHT, config.SPECTROGRAM_WIDTH),
    batch_size=config.AUDIO_TRAINING_BATCH_SIZE,
    classes=config.AUDIO_CLASS_NAMES,
    class_mode='sparse')

validation_generator = data_generator.flow_from_directory(
    config.SPECTROGRAM_VALIDATION_DIR,
    target_size=(config.SPECTROGRAM_HEIGHT, config.SPECTROGRAM_WIDTH),
    batch_size=config.AUDIO_VALIDATION_BATCH_SIZE,
    classes=config.AUDIO_CLASS_NAMES,
    class_mode='sparse',
    shuffle=False)

test_generator = data_generator.flow_from_directory(
    config.SPECTROGRAM_TEST_DIR,
    target_size=(config.SPECTROGRAM_HEIGHT, config.SPECTROGRAM_WIDTH),
    batch_size=config.AUDIO_TEST_BATCH_SIZE,
    classes=config.AUDIO_CLASS_NAMES,
    class_mode='sparse',
    shuffle=False)

Found 1491 images belonging to 4 classes.
Found 381 images belonging to 4 classes.
Found 175 images belonging to 4 classes.


# Train CNN Model

In [0]:
input_shape = (config.SPECTROGRAM_HEIGHT, config.SPECTROGRAM_WIDTH, config.CHANNELS)
dense_net_model = DenseNet121(include_top=False, input_shape=input_shape, pooling='avg')
x = dense_net_model.output
x = Dropout(0.5)(x)
x = Dense(config.N_AUDIO_CLASSES, name='audio_predictions')(x)
output = Softmax()(x)

model = Model(inputs=dense_net_model.input, outputs=output)
model.compile(optimizer=Adam(lr=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [0]:
early_stopping = EarlyStopping(patience=5, restore_best_weights=True, verbose=1)
model.fit_generator(
    training_generator,
    steps_per_epoch=len(training_generator),
    epochs=100,
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks=[early_stopping])

In [0]:
# recompiling the model will reduce the size of the saved model by removing the training related information
model.compile(optimizer=Adam(lr=0.00001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.save(config.AUDIO_MODEL_FILEPATH)

# Evaluate Model

In [0]:
model = load_model(config.AUDIO_MODEL_FILEPATH)

## Validation Set

In [21]:
y_true = validation_generator.classes
predictions = model.predict_generator(validation_generator, steps=len(validation_generator))
y_pred = np.argmax(predictions, axis=1)

report = classification_report(y_true, y_pred, target_names=config.AUDIO_CLASS_NAMES)
print(report)
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

         air       0.82      0.90      0.85       164
        fail       0.57      0.59      0.58        41
       grind       0.66      0.67      0.66        88
       slide       0.67      0.52      0.59        88

    accuracy                           0.72       381
   macro avg       0.68      0.67      0.67       381
weighted avg       0.72      0.72      0.72       381

[[147   5   6   6]
 [  9  24   4   4]
 [ 11   5  59  13]
 [ 13   8  21  46]]


## Test Set

In [20]:
y_true = test_generator.classes
predictions = model.predict_generator(test_generator, steps=len(test_generator))
y_pred = np.argmax(predictions, axis=1)

report = classification_report(y_true, y_pred, target_names=config.AUDIO_CLASS_NAMES)
print(report)
print(confusion_matrix(y_true, y_pred))

              precision    recall  f1-score   support

         air       0.60      0.48      0.53        50
        fail       0.33      0.04      0.07        25
       grind       0.52      0.64      0.57        50
       slide       0.33      0.46      0.38        50

    accuracy                           0.46       175
   macro avg       0.44      0.41      0.39       175
weighted avg       0.46      0.46      0.44       175

[[24  2  4 20]
 [ 4  1  8 12]
 [ 3  0 32 15]
 [ 9  0 18 23]]
